# **AC on/off notification**

**Introduction:**

As part of a study of the effect of green walls on interior spaces, our air conditioners in the laboratory are constantly on at 25 degrees Celsius.
But there are times when the air conditioners stop working. The AC status should be documented, since it affects the activity of the plants, and other results, etc.
We wrote a code that saves an image from a USB camera that sees the air conditioners and is connected to our server. The code identifies according to the vents of the air conditioner whether it is open or closed, that is, on or off. Accordingly, the status of the air conditioner is recorded with the exact time in a database, an updated graph is created, and a notification email is sent if there has been a change since the last check.

The image is processed by counting the number of black pixels, and comparing the ranges corresponding to the status of the air conditioner: on/off. Currently, the model works during the hours when there is lighting in the laboratory (natural or artificial).

To continue - inserting the images that are saved throughout the day into a machine learning model, in order to get predictions about an open/closed air conditioner under different lighting conditions.

You are welcome to take a look and use code parts if relevant to you!

**Pipeline:**

1. Capture image from webcam.
2. Cut polygon of vents area, and see if open or close.
3. Append status+timestamp to csv file.
4. CSV to df, and create plotly graph.
5. Send email with smtp.


**By:** Yedidya Harris, Yehuda Yungstein

### Libs

In [ ]:
# crop image by specific coords

import cv2
import numpy as np
from skimage import data
from skimage.viewer import ImageViewer
from skimage import data, io
from matplotlib import pyplot as plt
from skimage.color.colorconv import ycbcr2rgb
import os
import re
from datetime import datetime
from PIL import Image 
import PIL 

# global vars
folder_path = 'D:/LoggerNet-code'

## Capture image from USB webcam

In [ ]:
# Python program to capture a single image
# using pygame library

# importing the pygame library
import pygame
import pygame.camera
import time

def captureImg():
  # initializing the camera
  pygame.camera.init()

  # make the list of all available cameras
  camlist = pygame.camera.list_cameras()


  # if camera is detected or not
  if camlist:

    # initializing the cam variable with default camera
    cam = pygame.camera.Camera(camlist[0], (640, 480))

    # opening the camera
    cam.start()
    
    time.sleep(5) # letting the camera gain some light 

    # capturing the single image
    image = cam.get_image()
    current_date_time = datetime.now().strftime("%Y-%m-%d_%H_%M_%S") # current timestamp

    # saving the image
    image_path = f'{folder_path}/images/{current_date_time}_lab.jpg'
    pygame.image.save(image, f'{folder_path}/images/{current_date_time}_lab.jpg')

  # if camera is not detected the moving to else part
  else:
    print("No camera on current device")
  return image_path


## Image Processing + update csv

### threshold

In [ ]:
# a few tweaks

# threshold, indication for ac on/of - YEHUDA

#function to crop an image by xy coords (top left corner, bottom right corner)

def crop_image(image_path, x1,y1,x2,y2):
  img = cv2.imread(image_path)
  crop_img = img[y1:y2, x1:x2 ]
  return crop_img

# function to crop both ac vents, by specific coords
def crop_ac_vents(image_path):
  ac_left_img = crop_image(image_path, 420,840,635,910)
  ac_right_img = crop_image(image_path, 1150,535,1500,670)
  return ac_left_img, ac_right_img

# threshold, check if vents are open or closed
def get_ac_status (image_path):
  ac_left_img, ac_right_img = crop_ac_vents(image_path)

  # right:
  ac_right = ac_right_img # location of right ac
  img_thresh = np.where(ac_right < 110 , ac_right, 255) # above 110 all values wiil be 255
  img_thresh = np.where(img_thresh >= 110 , img_thresh, 0) # bebeath  110 all values wiil be 255
  #imgplot = plt.imshow(img_thresh) #plot
  pixels = np.sum(np.array(img_thresh) == 0) # how much pixels get the value 0
  if pixels >= 9900: #it means the ac on
    status_r = 1
    status_r_str = "ON"
  else:
    status_r = 0 #it means the ac off
    status_r_str = "OFF"
  
  # left - same procces different threshold
  ac_left = ac_left_img
  img_thresh = np.where(ac_left < 110 , ac_left, 255)
  img_thresh = np.where(img_thresh >= 110 , img_thresh, 0) 
  #imgplot = plt.imshow(img_thresh)
  pixels = np.sum(np.array(img_thresh) == 0)
  if pixels >= 1300:
    status_l = 1
    status_l_str = "ON"
  else:
   status_l = 0
   status_l_str = "OFF"

  # save cropped images with status
  current_date_time = datetime.now().strftime("%Y-%m-%d_%H_%M_%S") # current timestamp
  left_filename = f'{current_date_time}_left_{status_l}.jpg'
  right_filename = f'{current_date_time}_right_{status_r}.jpg'
  # cv2.imwrite(f'{folder_path}/images/cropped/{left_filename}', ac_left)
  # cv2.imwrite(f'{folder_path}/images/cropped/{right_filename}', ac_right)
  # print(os.listdir(f'{folder_path}/images/cropped'))
  
  return status_l,status_l_str,status_r,status_r_str

### update csv

In [ ]:
# add status to csv

# Importing required modules
from csv import writer
from datetime import datetime

def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

# function to log status of AC to csv
def log_status (leftac_status, rightac_status):
  current_date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S") # current timestamp
  file_name = f'{folder_path}/ac_status.csv' # our csv
  row_contents = [current_date_time, leftac_status, rightac_status] # new row to append
  append_list_as_row(file_name, row_contents) # append new row to csv file

## Send notification to email



In [ ]:
import smtplib
import ssl

# creds and settings
sender = 'sender@gmail.com'
recipient = ['mail1@example.com','mail2@example.com']
sender_password = 'secret'
port = 587  
smtp_server = "smtp.gmail.com"


def notifyEmail(left_ac_status_str,right_ac_status_str):
  message = f"""Subject: Lab AC STATUS


  Left AC status: {left_ac_status_str}. 
  Right AC status: {right_ac_status_str}.

  Good luck!
  """

  SSL_context = ssl.create_default_context()

  with smtplib.SMTP(smtp_server, port) as server:

      server.starttls(context=SSL_context)

      server.login(sender, sender_password)

      server.sendmail(sender, recipient, message)


## Check if AC status changed from last check

In [1]:
def check_notification(df,colomn_left_name,column_right_name):
  # check the last value in both columns and compare to value before it
  columns = [colomn_left_name,column_right_name]
  for column in columns:
    last_value = df[column].iat[-1]
    one_before = df[column].iat[-2]
    if last_value == one_before:
      send_mail = False
    else:
      send_mail = True
    return send_mail

## Reading data and plotting

In [ ]:
# function to plot a graph in plotly

import plotly.graph_objects as go
def plot_df (latest_df, col_x, col_y1, col_y2='none', trace1_name='none', trace2_name='none', x_title='', y_title='', plot_title=''):

  # Create figure
  fig = go.Figure()

  if col_y2 == 'none':   
      fig.add_trace(
          go.Scatter(x=list(latest_df[col_x]), y=list(latest_df[col_y1]),        
          
          hovertemplate="%{y}%{_xother}"
           ))

  if col_y2 != 'none':   
      fig.add_trace(
          go.Scatter(
              x=list(latest_df[col_x]), y=list(latest_df[col_y1]),
              name=trace1_name, hovertemplate="%{y}%{_xother}"      
          ))
      
      fig.add_trace(
          go.Scatter(
              x=list(latest_df[col_x]), y=list(latest_df[col_y2]),
              name=trace2_name, hovertemplate="%{y}%{_xother}"     
          ))

  # Set title
  fig.update_layout(
      yaxis={"dtick":1},
      xaxis_title=x_title,
      yaxis_title=y_title,

      height=600, width=1500,
      title={
      'text': plot_title,
      'y':0.9,
      'x':0.5,
      'xanchor': 'center',
      'yanchor': 'top'}
  )      
  

  # Add range slider
  fig.update_layout(
      xaxis=dict(
          rangeselector=dict(
              buttons=list([
                  dict(count=1,
                      label="1minute  ",
                      step="minute",
                      stepmode="backward"),            
                  dict(count=1,
                      label="1hour",
                      step="hour",
                      stepmode="backward"),       
                  dict(count=1,
                      label="1day",
                      step="day",
                      stepmode="backward"),                 
                  dict(count=1,
                      label="1month",
                      step="month",
                      stepmode="backward"),
                  dict(count=6,
                      label="6months",
                      step="month",
                      stepmode="backward"),
                  dict(step="all")
              ])
          ),
          rangeslider=dict(
              visible=True
          ),
          type="date"
      )
  )

  fig.update_layout(hovermode="x unified")

  return fig    

In [ ]:
import pandas as pd
def graph_to_html():
  file_name = f'{folder_path}/ac_status.csv' # our csv
  df = pd.read_csv(file_name)
  df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], format='%Y-%m-%d %H:%M:%S') 
  #df=df.set_index('TIMESTAMP')
  ac_fig = plot_df (df, 'TIMESTAMP', 'LEFT_STATUS', col_y2='RIGHT_STATUS', trace1_name='Left AC', trace2_name='Right AC', x_title='TIMESTAMP', y_title='On=1 | Off=0', plot_title='AC Status')

  # save plotly graph to html   
  html_path = f'{folder_path}/ac_status.html'
  with open(html_path, 'w') as f: # w for overwrite
    f.write(ac_fig.to_html(full_html=False, include_plotlyjs='cdn'))
  return df

## Running our app in a loop

In [ ]:
# set a loop, run every 10 minutes
import glob

def runAll():
  #images_path = glob.glob(f'{folder_path}/images/*.jpg') # get a list of all the paths of the images within the folder
  
  image_path = captureImg() # captures an img, saves, and returns the path
  leftac_status, left_ac_status_str, rightac_status, right_ac_status_str = get_ac_status(image_path) # get ac status
  log_status (leftac_status, rightac_status) # log ac status to csv  
  df = graph_to_html() # plot the csv and save to html, and return df
  
  #check if AC status changed
  if check_notification(df,'LEFT_STATUS','RIGHT_STATUS'): 
    notifyEmail(left_ac_status_str,right_ac_status_str) # if left/right ac off, then send email

In [ ]:
# run function every 5 minutes
import time

def logAc():
  if __name__ == '__main__':
      while True:
          runAll()
          print('AC status logged.')
          time.sleep(5*60) # run evey 5 minutes

In [ ]:
logAc()